# Regression Exercise 

California Housing Data

This data set contains information about all the block groups in California from the 1990 Census. In this sample a block group on average includes 1425.5 individuals living in a geographically compact area. 

The task is to aproximate the median house value of each block from the values of the rest of the variables. 

 It has been obtained from the LIACC repository. The original page where the data set can be found is: http://www.liaad.up.pt/~ltorgo/Regression/DataSets.html.
 

The Features:
 
* housingMedianAge: continuous. 
* totalRooms: continuous. 
* totalBedrooms: continuous. 
* population: continuous. 
* households: continuous. 
* medianIncome: continuous. 
* medianHouseValue: continuous. 

## The Data

** Import the cal_housing_clean.csv file with pandas. Separate it into a training (70%) and testing set(30%).**

In [2]:
import pandas as pd

In [6]:
housing = pd.read_csv('cal_housing_clean.csv', sep=',',)
housing.drop('Unnamed: 0', axis=1, inplace=True) 

In [7]:
housing.head()

,HouseAge,AveRooms,AveBedrms,Population,AveOccup,MedInc,Latitude,Longitude,medianHouseValue
0,41.0,6.984127,1.023810,322.0,2.555556,8.3252,37.88,-122.23,4.526
1,21.0,6.238137,0.971880,2401.0,2.109842,8.3014,37.86,-122.22,3.585
2,52.0,8.288136,1.073446,496.0,2.802260,7.2574,37.85,-122.24,3.521
3,52.0,5.817352,1.073059,558.0,2.547945,5.6431,37.85,-122.25,3.413
4,52.0,6.281853,1.081081,565.0,2.181467,3.8462,37.85,-122.25,3.422


In [8]:
y = housing['medianHouseValue']

In [9]:
X = housing.drop(['medianHouseValue'], axis=1)

In [16]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)

### Scale the Feature Data

** Use sklearn preprocessing to create a MinMaxScaler for the feature data. Fit this scaler only to the training data. Then use it to transform X_test and X_train. Then use the scaled X_test and X_train along with pd.Dataframe to re-create two dataframes of scaled data.**

In [17]:
from sklearn.preprocessing import MinMaxScaler

In [18]:
scaler = MinMaxScaler()
scaler.fit(X_train)

MinMaxScaler(copy=True, feature_range=(0, 1))

In [19]:
X_train = pd.DataFrame(scaler.transform(X_train), columns = X_train.columns, index=X_train.index)
X_test = pd.DataFrame(scaler.transform(X_test), columns = X_test.columns, index=X_test.index)

In [20]:
X_train.head()
X_test.head()

,HouseAge,AveRooms,AveBedrms,Population,AveOccup,MedInc,Latitude,Longitude
8914,0.745098,0.024758,0.019658,0.030914,0.004052,0.181039,0.157447,0.582915
2847,0.431373,0.029699,0.020132,0.074677,0.002883,0.219831,0.303191,0.533668
2007,0.627451,0.020192,0.022846,0.123481,0.005810,0.042972,0.446809,0.449246
8267,0.941176,0.025419,0.020978,0.045899,0.001746,0.228452,0.129787,0.613065
17172,0.529412,0.044486,0.021688,0.014424,0.003343,0.507814,0.513830,0.201005


### Create Feature Columns

** Create the necessary tf.feature_column objects for the estimator. They should all be trated as continuous numeric_columns. **

In [21]:
import tensorflow as tf

In [25]:
HouseAge = tf.feature_column.numeric_column('HouseAge')
AveRooms = tf.feature_column.numeric_column('AveRooms')
AveBedrms = tf.feature_column.numeric_column('AveBedrms')
Population = tf.feature_column.numeric_column('Population')
AveOccup = tf.feature_column.numeric_column('AveOccup')
MedInc = tf.feature_column.numeric_column('MedInc')
Longitude = tf.feature_column.numeric_column('Longitude')
Latitude = tf.feature_column.numeric_column('Latitude')

** Create the input function for the estimator object. (play around with batch_size and num_epochs)**

In [28]:
input_func = tf.estimator.inputs.pandas_input_fn(x = X_train, y = y_train, batch_size=10, num_epochs=100, shuffle=True)

** Create the estimator model. Use a DNNRegressor. Play around with the hidden units! **

In [27]:
model = tf.estimator.DNNRegressor(hidden_units=[10, 10, 10, 10, 10], feature_columns=[HouseAge, AveRooms, AveBedrms, Population, AveOccup, MedInc, Longitude, Latitude])

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\user\\AppData\\Local\\Temp\\tmph9vzkqbg', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000001546059B710>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


##### ** Train the model for ~1,000 steps. (Later come back to this and train it for more and check for improvement) **

In [29]:
model.train(input_fn = input_func, steps = 1000)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into C:\Users\user\AppData\Local\Temp\tmph9vzkqbg\model.ckpt.
INFO:tensorflow:loss = 82.5778, step = 1
INFO:tensorflow:global_step/sec: 158.221
INFO:tensorflow:loss = 9.014645, step = 101 (0.642 sec)
INFO:tensorflow:global_step/sec: 202.955
INFO:tensorflow:loss = 3.6478677, step = 201 (0.501 sec)
INFO:tensorflow:global_step/sec: 196.958
INFO:tensorflow:loss = 4.176824, step = 301 (0.494 sec)
INFO:tensorflow:global_step/sec: 263.872
INFO:tensorflow:loss = 6.826818, step = 401 (0.389 sec)
INFO:tensorflow:global_step/sec: 279.509
INFO:tensorflow:loss = 6.2799287, step = 501 (0.349 sec)
INFO:tensorflow:global_step/sec: 275.639
INFO:tensorflow:loss = 7.9279094, step = 601 (0.366 sec)
INFO:tensorflow:global_step/sec: 2

** Create a prediction input function and then use the .predict method off your estimator model to create a list or predictions on your test data. **

In [30]:
pred_input_func = tf.estimator.inputs.pandas_input_fn(x = X_test, batch_size=10, num_epochs=1, shuffle=True)

In [32]:
pred_list = list(model.predict(pred_input_func))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\user\AppData\Local\Temp\tmph9vzkqbg\model.ckpt-1000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [33]:
predicts = []
for pred in pred_list:
    predicts.append(pred['predictions'])
predicts

[array([2.456462], dtype=float32),
 array([2.519335], dtype=float32),
 array([2.6870778], dtype=float32),
 array([2.276994], dtype=float32),
 array([3.2098918], dtype=float32),
 array([0.90038157], dtype=float32),
 array([4.255417], dtype=float32),
 array([4.539957], dtype=float32),
 array([1.9011955], dtype=float32),
 array([4.911498], dtype=float32),
 array([1.4612341], dtype=float32),
 array([2.0009942], dtype=float32),
 array([2.827391], dtype=float32),
 array([0.90550447], dtype=float32),
 array([2.9658751], dtype=float32),
 array([1.5509789], dtype=float32),
 array([3.3398523], dtype=float32),
 array([3.1833622], dtype=float32),
 array([4.882604], dtype=float32),
 array([0.9686762], dtype=float32),
 array([1.0873535], dtype=float32),
 array([2.6250393], dtype=float32),
 array([1.6776108], dtype=float32),
 array([1.259662], dtype=float32),
 array([2.6194057], dtype=float32),
 array([1.65143], dtype=float32),
 array([1.6101067], dtype=float32),
 array([2.0436406], dtype=float32),
 

** Calculate the RMSE. You should be able to get around 100,000 RMSE (remember that this is in the same units as the label.) Do this manually or use [sklearn.metrics](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_squared_error.html) **

In [34]:
from sklearn.metrics import mean_squared_error
import math

In [35]:
rmse = math.sqrt(mean_squared_error(y_test, predicts))
rmse

1.4526003385922943

# Great Job!